# 프로젝트명: 데이터 수집 실습 2 (공공데이터포털 API로 받기) 

#### 📌 requests는 API를 호출할 때 주로 사용하는 라이브러리입니다. 
- 서버에 HTTP 요청을 보낼때 주로 사용하며, GET, POST, PUT, DELETE를 사용할 수 있습니다.
- 데이터 API의 요구사항에 맞게 요청해야하며, 보통은 "GET", "POST"를 주로 사용됩니다.

#### 🚨 주의사항
- 공공데이터포털에선 운영계정을 신청해서 서비스키를 받아야합니다.
- 자세한 내용은 보통 API 사용문서를 참고합니다. 

![dddd](https://file.notion.so/f/f/4e4750b9-2973-47f2-9fe5-8330ce1f2c19/b501deab-8d96-4c0a-aca3-eb13441bb80a/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA_2024-05-22_%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE_9.00.51.png?id=bcc828ed-b1c4-4818-b658-4604e9ba6c26&table=block&spaceId=4e4750b9-2973-47f2-9fe5-8330ce1f2c19&expirationTimestamp=1716472800000&signature=TxFSv1p8bqyg9WZNNQZUmgFX78b1_1cSuwGJ75amj9o&downloadName=%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA+2024-05-22+%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE+9.00.51.png)

### 1일 차이의 사이 값 가져오기

In [1]:
from datetime import datetime, timedelta

start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-01-01', '%Y-%m-%d')

date_range = []
for i in range((end_date - start_date).days + 1):
    res_date = start_date + timedelta(days=i)
    res_date = res_date.strftime('%Y-%m-%d')
    date_range.append(res_date)

### 1년 차이의 한달 간격 가져오기 

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-01-01', '%Y-%m-%d')

date_range = []
current_date = start_date
while current_date < end_date:
    res_date = current_date.strftime('%Y-%m-%d')
    date_range.append(res_date)
    current_date += relativedelta(months=1)

print(date_range)

['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01']


### 기상청의 2023년 서울의 데이터 가져오기 (1년치 데이터 가져오기)

In [3]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-02-01', '%Y-%m-%d')

date_range = []
current_date = start_date
while current_date < end_date:
    res_date = current_date.strftime('%Y%m%d')
    date_range.append(res_date)
    current_date += relativedelta(months=1)

print(date_range)

['20230101', '20230201', '20230301', '20230401', '20230501', '20230601', '20230701', '20230801', '20230901', '20231001', '20231101', '20231201', '20240101']


In [4]:
start_end_date = []
for i in range(len(date_range) -1):
    start_end_date.append([date_range[i], date_range[i+1]])

In [11]:
from tqdm import tqdm
import requests
import pandas as pd
import time

url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
key = '''LesYUIlTmuQq1eV6uz8dCZpKRnR99hlOovQyAXmy12+rySvQVRlT1g0k/icLiruFYRK7qiat0jD23pR/xxSwWQ=='''

# https://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?
# serviceKey=LesYUIlTmuQq1eV6uz8dCZpKRnR99hlOovQyAXmy12%2BrySvQVRlT1g0k%2FicLiruFYRK7qiat0jD23pR%2FxxSwWQ%3D%3D
# &pageNo=1
# &numOfRows=10
# &dataType=XML
# &dataCd=ASOS
# &dateCd=HR
# &startDt=20100101
# &startHh=01
# &endDt=20100601
# &endHh=01&stnIds=108
dfs = []
for i in tqdm(start_end_date):
    print(i)

    params = {'serviceKey' : key, 
            'pageNo' : '1', 
            'numOfRows' : '999', 
            'dataType' : 'JSON', 
            'dataCd' : 'ASOS', ## 자료분류코드
            'dateCd' : 'HR', ## 날짜분류코드
            'startDt' : str(i[0]), ## 시작일
            'startHh' : '00', ## 시작시간
            'endDt' : str(i[1]), ## 종료일
            'endHh': '00', ## 종료시간
            'stnIds' : '108' ## 서울지역코드
            }

    response = requests.get(url, params=params)
    result = response.json()
    df = pd.DataFrame(result['response']['body']['items']['item'])
    dfs.append(df)

final_df = pd.concat(dfs, axis=0, ignore_index=True)

  0%|          | 0/12 [00:00<?, ?it/s]

['20230101', '20230201']


  8%|▊         | 1/12 [00:04<00:48,  4.43s/it]

['20230201', '20230301']


 17%|█▋        | 2/12 [00:08<00:45,  4.50s/it]

...

 83%|████████▎ | 10/12 [01:03<00:13,  6.68s/it]

['20231101', '20231201']


 92%|█████████▏| 11/12 [01:13<00:07,  7.94s/it]

['20231201', '20240101']


100%|██████████| 12/12 [01:19<00:00,  6.61s/it]


In [12]:
result

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'tm': '2023-12-01 00:00',
      'rnum': '1',
      'stnId': '108',
      'stnNm': '서울',
      'ta': '-5.0',
      'taQcflg': '',
      'rn': '',
      'rnQcflg': '',
      'ws': '2.1',
      'wsQcflg': '',
      'wd': '250',
      'wdQcflg': '',
      'hm': '64',
      'hmQcflg': '',
      'pv': '2.7',
      'td': '-10.7',
      'pa': '1016.3',
      'paQcflg': '',
      'ps': '1027.4',
      'psQcflg': '',
      'ss': '',
      'ssQcflg': '9',
      'icsr': '',
      'dsnw': '',
      'hr3Fhsc': '',
      'dc10Tca': '0',
      'dc10LmcsCa': '0',
      'clfmAbbrCd': '',
      'lcsCh': '',
      'vs': '3354',
      'gndSttCd': '',
      'dmstMtphNo': '',
      'ts': '-3.7',
      'tsQcflg': '',
      'm005Te': '1.5',
      'm01Te': '1.8',
      'm02Te': '2.8',
      'm03Te': '4.3'},
     {'tm': '2023-12-01 01:00',
      'rnum': '2',
      'stnId': '108',
   

In [13]:
final_df

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,2023-01-01 00:00,1,108,서울,0.9,,,9,1.4,,...,,1503,,,-1.6,,-0.3,-0.6,-0.5,0.6
1,2023-01-01 01:00,2,108,서울,1.5,,,,1.9,,...,12,1104,,,-1.4,,-0.3,-0.5,-0.5,0.6
2,2023-01-01 02:00,3,108,서울,1.5,,,,1.9,,...,11,925,,40,-1.6,,-0.3,-0.5,-0.5,0.6
3,2023-01-01 03:00,4,108,서울,1.6,,,,1.6,,...,16,794,,40,-1.8,,-0.3,-0.5,-0.5,0.6
4,2023-01-01 04:00,5,108,서울,1.5,,,,1.4,,...,10,709,,40,-1.3,,-0.2,-0.5,-0.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8767,2023-12-31 20:00,741,108,서울,1.9,,,9,2.2,,...,,716,,19,-0.2,,-0.1,-0.2,0.0,1.0
8768,2023-12-31 21:00,742,108,서울,1.7,,,9,2.4,,...,,522,,19,-0.2,,-0.1,-0.2,0.0,1.0
8769,2023-12-31 22:00,743,108,서울,1.3,,,9,0.6,,...,,423,,19,-0.2,,-0.1,-0.2,0.0,1.0
8770,2023-12-31 23:00,744,108,서울,0.9,,,9,0.8,,...,,656,,19,-0.2,,-0.1,-0.2,0.0,1.0
